## 特征工程

In [1]:
# 数据和特征决定了机器学习的上限，而模型和算法只是逼近这个上限而已。 ⚠️重点！数据的数量和质量比算法重要的多！
# 特征工程包括：
# 1.特征使用 （关注点都在特征的原数据上）
    # 数据的选择：是分析和我们目标最相关的数据都有哪些，这些数据如何获取。
    # 数据的可用性判断：是指数据特征是否可以持续输出
# 2.特征获取
    # 特征来源：即我们需要的特征来自与哪张表或哪个文件
    # 特征存储：如特征来自于不同的文件或不同的数据库，就要将数据进行规整，存储在将来想要使用的媒介中
# 3.特征处理：目的是数据属性和特征尽可能大的发挥作用，体现差别
    # 数据清洗：
    # 特征预处理：
# 4.特征监控
    # 现有特征：主要看是否对现有的任务还存在积极的作用
    # 新的特征：看知否有助于提高效果，或更能代表我们的任务目标

#### 数据清洗

In [2]:
# 数据样本抽样
# 样本要具有代表性
# 样本比例平衡以及样本不平衡时如何处理
# 考虑全量数据 （hadoop、spark大数据工具）

In [3]:
# 异常值（空值）处理
# 识别异常值和重复值  pandas:isnull()/duplicated()
# 直接丢弃（包括重复数据） pandas:drop()/dropna()/drop_duplicated()
# 将异常值当作一个新的属性，替代原值 pandas:fillna()
# 集中值指代：pandas:fillna() 集中值可以是：除异常值外的均值、中位数、众数等
# 边界值指代：pandas:fillna() 利用四分位数的边界
# 插值： pandas:interpolate() --- Series

In [2]:
# 字符类型的空值是：None
# 数值类型的空值是：NaN
df = [[],[]]
# 查看空值
df.isnull()
# 删除空值
df.dropna()
# 去掉B行的空值
df.dropna(subset=["B"])
# 识别重复值
df.duplicated(["A"])
# 删除重复值
# keep="first" last, False:删除所有重复值
# inplace=True  可以让DF的index也跟着发生变化
df.drop_duplicates(["A"], keep="first")
# 标注异常值
df.fillna("b*")
# 集中值指代 均值
df.fillna(df["E"].mean())
# 插值 （只能用于series数据），返回的是相邻两个值的平均值，处于两端的话，就取最近的数
# method="spline", order=3  三次样条
df["E"].interpolate()
# 用四分位数确定上下界的方法进行过滤
upper_q = df["D"].quantile(0.75)
lower_q = df["D"].quantile(0.25)
q_int = upper_q - lower_q
k = 1.5
df[df["D"]>lower_q-k*q_int][df["D"]<upper_q+k*q_int]
# 去除异常值
df.drop(2)
# 或者
df[[True if item.startswith("f") else False for item in list(df["F"].values)]]

AttributeError: 'list' object has no attribute 'isnull'

#### 标注

In [3]:
# 标注（标记、标签、label）

#### 特征预处理

In [5]:
# 特征选择：剔除与标注不相关或者冗余的特征，减少特征的个数。作用：减少了训练的时间，减少了过拟合。是数据规约的一种处理方式（另一个为抽样）
    # 特征选择有三个切入思路；
    # 1.过滤思想：就是直接评价某个特征与标注的相关性的特征，如果与标注的相关性非常小就去掉。（复习下面的特征选择表）
    # 2.包裹思想：遍历特征子集。假设所有的集合是一个集合X，最佳的特征组合是它的一个子集，我们的任务就是要找到这个子集。
    # RFE算法：
    # 第一步：列出集合X。
    # 第二步：构造简单的模型进行训练，根据系数去掉比较弱的特征。
    # 第三部：余下的特征重复这个过程，直到评价指标下降较大或者低于阀值，停止
    # 3.嵌入思想：建立简单的回归模型。根据一个简单的模型来分析特征的重要性。最常用的方式是用正则化的方式来做特征选择。
# 特征变换：对指化、离散化、数据平滑、归一化（标准化）、数值化、正规化
# 特征降维 
# 特征衍生 

#### 特征选择表

In [6]:
#    数据类型                           可用方法
# 连续----连续                       相关系数、假设检验
# 连续----离散（二值）                相关系数、连续二值化（最小Gini切分，最大熵增益切分）
# 连续----离散（非二值）              相关系数（定序）
# 离散（二值）----离散（二值）         相关系数、熵相关、F分值
# 离散----离散（非二值）              熵相关、Gini、相关系数（定序）

#### 代码实现

In [7]:
import numpy as np
import pandas as pd
import scipy.stats as ss

In [16]:
# 生成一组数据
df = pd.DataFrame({"A":ss.norm.rvs(size=10),
               "B":ss.norm.rvs(size=10),
               "C":ss.norm.rvs(size=10),
               "D":np.random.randint(low=0, high=2, size=10)})

In [17]:
df

,A,B,C,D
0,0.068055,-0.443143,1.108843,1
1,1.660826,-0.154182,-1.728332,1
2,0.075818,-0.223309,0.407641,1
3,-1.089937,-0.967766,-1.140794,1
4,-1.022013,0.151264,-0.160691,1
5,0.086538,0.258648,-1.856277,0
6,-1.424559,-0.083167,1.027530,1
7,-0.627077,1.153571,-1.524347,0
8,-0.850279,-0.743631,0.424328,0
9,-0.427147,-1.664223,1.348109,1


In [18]:
# 引入SVR回归器、
# DecisionTreeRegressor决策树回归器
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

In [21]:
# 确定特征和标注
# 特征
X = df.loc[:,["A", "B", "C"]]
# 标注
Y = df.loc[:, "D"]

In [23]:
# 特征选择常用的三个类
# 过滤思想 SelectKBest
# 包裹思想 RFE
# 嵌入思想 SelectFromModel
from sklearn.feature_selection import SelectKBest, RFE, SelectFromModel

#### 过滤思想

In [25]:
# f_classif 通过方差分析的F值进行判定的（默认的）
# mutual_info_classif 互信息
# chi2 卡方校验
skb = SelectKBest(k=2)
skb.fit(X,Y)
skb.transform(X)

array([[-0.44314267,  1.10884264],
       [-0.15418172, -1.72833239],
       [-0.22330872,  0.40764105],
       [-0.96776614, -1.14079445],
       [ 0.15126358, -0.16069073],
       [ 0.25864802, -1.85627674],
       [-0.08316678,  1.02752952],
       [ 1.15357136, -1.52434747],
       [-0.74363087,  0.424328  ],
       [-1.66422293,  1.3481086 ]])

#### 包裹思想 RFE

In [26]:
# SVR 线性回归器 
# n_features_to_select 最终要选择的特征数
# step 每一步要去掉多少个特征  step=1每迭代一次去掉一个特征 
rfe = RFE(estimator=SVR(kernel="linear"), n_features_to_select=2, step=1)
# fit_transform 拟合过后再进行变换
rfe.fit_transform(X, Y)

array([[ 0.06805487, -0.44314267],
       [ 1.66082604, -0.15418172],
       [ 0.07581797, -0.22330872],
       [-1.08993669, -0.96776614],
       [-1.02201321,  0.15126358],
       [ 0.08653789,  0.25864802],
       [-1.42455855, -0.08316678],
       [-0.62707671,  1.15357136],
       [-0.85027892, -0.74363087],
       [-0.42714716, -1.66422293]])

#### 嵌入思想 SelectFromModel

In [27]:
# threshold 表示重要性因子的数（低于多少就会被去掉）
sfm = SelectFromModel(estimator=DecisionTreeRegressor(), threshold=0.1)
sfm.fit_transform(X, Y)

array([[-0.44314267],
       [-0.15418172],
       [-0.22330872],
       [-0.96776614],
       [ 0.15126358],
       [ 0.25864802],
       [-0.08316678],
       [ 1.15357136],
       [-0.74363087],
       [-1.66422293]])

### 特征变换

In [28]:
# 特征变换：就是根据特征的特性进行一定方式的转换，使特征能够发挥出它的特点
# 特征变换的方法：
# 1.对指化：对数据进行对数化和指数化的过程  使用函数 softmax 即可完成  numpy.exp
# 2.对数化：对数据取对数的过程，底可以取e、10、2  例如月收入情况  numpy.log

### 离散化

In [1]:
# 将连续变量分成几段（bins）,变成离散数据的选择。
# 原因：
# 1.克服数据缺陷
# 2.某些算法要求
# 3.非线数据映射
# 方法：
# 数据分箱技术有：等频（等深分箱）、等距（等宽分箱）
# 自因变量优化：根据自变量、因变量的有序分布，找到拐点等特殊变化点进行离散化

#### 离散化（分箱）

In [2]:
# 数据在进行分箱前一定要先进行排序
# 深度：数据的个数
# 宽度：数据的区间 

In [6]:
import numpy as np
import pandas as pd
lst = [6, 8, 10, 15, 16, 24, 25, 40, 67]

In [8]:
# 等深分箱
# qcut 方法
# q 分成的份数
# labels 标注
pd.qcut(lst, q=3, labels=["low", "medium", "high"])

[low, low, low, medium, medium, medium, high, high, high]
Categories (3, object): [low < medium < high]

In [9]:
# 等宽分箱
# bins=3 分成3段
pd.cut(lst, bins=3)

[(5.939, 26.333], (5.939, 26.333], (5.939, 26.333], (5.939, 26.333], (5.939, 26.333], (5.939, 26.333], (5.939, 26.333], (26.333, 46.667], (46.667, 67.0]]
Categories (3, interval[float64]): [(5.939, 26.333] < (26.333, 46.667] < (46.667, 67.0]]

In [10]:
pd.cut(lst, bins=3, labels=["low", "medium", "high"])

[low, low, low, low, low, low, low, medium, high]
Categories (3, object): [low < medium < high]

### 归一化和标准化

#### 归一化

In [14]:
# 最大化、最小化的一种特殊形式。归一化将数据转化到 0-1 之间的范围。方法将每个数减去数据的最小值，然后除以最大值与最小值的差。
# 好处：处理起来会更方便一些，一方面可以直接观察单个数据相对于整体数据的比例，另一方面如果遇到不同样的数据特征，可以方便的建立起这些数据特征之间合适的距离度量方法。

#### 标准化

In [15]:
# 将数据转换成一个标准的形式。
# 狭义的理解：将数据缩放到均值为 0， 标准差为 1 的尺度上，这种转化也叫做韦德分标准化（Z-score）。
# 方法：一个特征的每个数减去它的均值，然后再除以它的标准差，就得到 Z-score 转化

In [17]:
# 引入归一化与标准化函数 (需要链接外网)
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [18]:
# 归一化
# reshape(-1, 1)  -1代表不指定有多少行， 1代表必须有一列
MinMaxScaler().fit_transform(np.array([1, 4, 10, 15, 21]).reshape(-1, 1))

array([[0.  ],
       [0.15],
       [0.45],
       [0.7 ],
       [1.  ]])

In [19]:
# 标准化
StandardScaler().fit_transform(np.array([1, 1, 1, 1, 0, 0, 0, 0]).reshape(-1, 1))

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [-1.],
       [-1.],
       [-1.],
       [-1.]])

In [20]:
StandardScaler().fit_transform(np.array([1,0, 0, 0, 0, 0, 0, 0]).reshape(-1, 1))

array([[ 2.64575131],
       [-0.37796447],
       [-0.37796447],
       [-0.37796447],
       [-0.37796447],
       [-0.37796447],
       [-0.37796447],
       [-0.37796447]])

### 数值化

In [2]:
# 数值化就是把非数值数据转化成数值数据的过程。
# 数据分类：
# 1.定类：将数据进行数值化，使其可以进行四则运算。独热（one-HotEncode）：将数据特征进行扩维，原来的N维属性，用N维向量数据来表示，这个向量只有一位是1，其它均为0.
# eg
# red  ->[1, 0, 0, 0]
# yellow  ->[0, 1, 0, 0]
# blue  ->[0, 0, 1, 0]
# green  ->[0, 0, 0, 1]

# 2.定序：将数据进行数值化，使其可以进行四则运算。通过标签化(labelEncoder)的方式 （0，1）
# 3.定距：可以通过归一化的方式，消除其物理含义，从而具备乘除的能力
# 4.定比

# 数值化的作用是为了以后建模时使用，不会将其直接相加减

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

/Applications/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Applications/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [4]:
# 标签化(labelEncoder)的方式
LabelEncoder().fit_transform(np.array(["Down", "Up", "Up", "Down"]).reshape(-1, 1))

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 1, 1, 0])

In [10]:
# 独热（one-HotEncode）
# 要先进行标签化
lb_encoder = LabelEncoder()
lb_tran_f = lb_encoder.fit_transform(np.array(["Red", "Yellow", "Blue", "Green"]))
oht_encoder = OneHotEncoder().fit(lb_tran_f.reshape(-1, 1))
oht_encoder.transform(lb_encoder.transform(np.array(["Yellow", "Blue", "Green", "Green", "Red"])).reshape(-1, 1)).toarray()

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.]])

### 正规化

In [11]:
# 是将一个向量的长度正规到单位1，如果距离尺度的衡量用L1距离，那就是L1正规化。如果用L2表示（欧式长度），就是L2正则化
# 方法：分子保持向量的分量不变，分母用的是各个向量绝对值的和。
# 用法：
# 1.直接用在特征上。如果正规化用在某一个特征上，可以将特征间的差距转化成考虑相对整体特征长度的一个相对值
# 2.用在每个对象的各个特征的表示（特征矩阵的行）。如果每个向量的分量都是一个特征，此时进行正规化，我们可以体现出一个对象特征之间影响的相对关系特点
# 3.模型的参数上（回归模型使用较多）。如线性回归、逻辑回归等。比如我们可以用L2正则化让所有的系数的平方和唯一，可以表示出哪些特征对于标注的影响占比比较大，哪些占比比较小。

In [15]:
from sklearn.preprocessing import Normalizer
# norm="l1" 选择方法 L1、L2
Normalizer(norm="l1").fit_transform(np.array([[1, 1, 3, -1, 2]]))

array([[ 0.125,  0.125,  0.375, -0.125,  0.25 ]])

### 特征降维

In [16]:
# PCA、奇异值分解等线性降维（没有用到标注）
# LDA降维（使用了标注）Linear Discriminant Analysis 线性判别式分析
# ⚠️不是：隐含狄利克雷分布（Latent Dirichlet Allocation）,主要用于自然语言处理中主题模型的建立。
# 核心思想：投影变换后同一标注内的距离尽可能小；不同标注间的距离尽可能大。

In [21]:
# LDA
import numpy as np
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [22]:
# solver下的参数   
# svd 奇异值分解
# lsqr 
# eigen 特征值

X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
Y = np.array([1, 1, 1, 2, 2, 2])
LinearDiscriminantAnalysis(n_components=1).fit_transform(X, Y)

array([[-1.73205081],
       [-1.73205081],
       [-3.46410162],
       [ 1.73205081],
       [ 1.73205081],
       [ 3.46410162]])

In [24]:
# 当作判别器使用 fisher classifi分类器
clf = LinearDiscriminantAnalysis(n_components=1).fit(X, Y)
clf.predict([[0.8, 1]])

array([2])

In [25]:
np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])

array([[-1, -1],
       [-2, -1],
       [-3, -2],
       [ 1,  1],
       [ 2,  1],
       [ 3,  2]])

### 特征衍生

In [ ]:
# 是现有的特征进行某些组合生成新的具有含义的特征
# 常用的方法有：
# 加减乘除
# 求导与高阶求导
# 人工归纳